# **Login Weights & Biases**

In [2]:
import wandb
wandb.login() # Login

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: ayenyeinsan2904 (ayenyeinsan2904-chiang-mai-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

# **Helper Functions:**

In [3]:
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

import joblib

def load_raw_titanc():
  # Load the Titanic dataset
  titanic_url = 'https://raw.githubusercontent.com/datasciencedojo/datasets/master/titanic.csv'
  return pd.read_csv(titanic_url)

def clean_titanic(titanic_data, fill_missing=True):
  df = titanic_data.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
  # Fill missing values
  if fill_missing:
    df['Age'].fillna(df['Age'].median(), inplace=True)
    df['Embarked'].fillna(df['Embarked'].mode()[0], inplace=True)
    df['Fare'].fillna(df['Fare'].median(), inplace=True)
  else:
    df = df.dropna()
  # Encode categorical features
  df['Sex'] = df['Sex'].map({'male': 0, 'female': 1})
  df['Embarked'] = df['Embarked'].map({'C': 0, 'Q': 1, 'S': 2})
  return df

def load_titanic(cleaned=True, Xy=True):
  '''
  Returns the titanic dataset
  '''
  # Get data
  df = load_raw_titanc()
  # Clean data
  if cleaned:
    # Drop useless features
    df = clean_titanic(df)
  # Split Data
  if Xy:
    return df.drop('Survived', axis=1), df['Survived'] # X, y
  return df

def get_data_loaders(X_train, X_test, y_train, y_test, batch_size=32):
  # Convert to Tensors
  X_train_tensor = torch.tensor(X_train.to_numpy(), dtype=torch.float32)
  y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
  X_test_tensor = torch.tensor(X_test.to_numpy(), dtype=torch.float32)
  y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)

  # Create Datasets
  train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
  test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

  # Create DataLoaders
  train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
  test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

  return train_loader, test_loader

def train_loop(train_loader, model, optimizer, criterion, device):
  model.train()
  epoch_train_loss = 0.0
  for batch, labels in train_loader:
    batch, labels = batch.to(device), labels.to(device)
    optimizer.zero_grad()
    # Forward Pass
    outputs = model(batch)
    loss = criterion(outputs, labels)
    # Update Parameters
    loss.backward()
    optimizer.step()
    # Save loss
    epoch_train_loss += loss.item() * batch.size(0)
  return epoch_train_loss / len(train_loader.dataset)

def test_loop(test_loader, model, criterion, device):
  model.eval()
  epoch_test_loss = 0.0
  with torch.no_grad():
    for batch, labels in test_loader:
      batch, labels = batch.to(device), labels.to(device)
      # Calculate predictions
      outputs = model(batch)
      # Save loss
      loss = criterion(outputs, labels)
      epoch_test_loss += loss.item() * batch.size(0)
  return epoch_test_loss / len(test_loader.dataset)

if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

print(device)

cpu


# **Simple Scikit-Learn Run**

In [11]:
# Define hyperparameters
n_estimators = 30
max_depth = 2

# Create a new run in a project
run = wandb.init(
    project="Simple Scikit-Learn Run",
    notes="commit message for the run",
    config={
        "n_estimators": n_estimators,
        "max_depth": max_depth
    }
)

# Get Data
X, y = load_titanic()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

# Define Random Forest Classifier
clf = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth)
clf.fit(X_train, y_train)

# Log metrics
wandb.log({"accuracy": clf.score(X_test, y_test)})

# Finish the run
wandb.finish()

<ipython-input-6-508fe56c84fc>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-6-508fe56c84fc>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

accuracy,▁
accuracy,0.78771


# **Simple PyTorch Run**

In [12]:
# Define hyperparameters
num_epochs = 10
learning_rate = 0.01
run_count = 3

# Start a run
run = wandb.init(
    project="Simple PyTorch Run",
    name=f"MyRun{run_count}",
    config={
        "num_epochs": num_epochs,
        "learning_rate": learning_rate
    }
)

# Get Data
X, y = load_titanic()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)
train_loader, test_loader = get_data_loaders(X_train, X_test, y_train, y_test)

# Define Simple PyTorch Model
model = nn.Sequential(
    nn.Linear(7, 32), # 7 input features -> 32 hidden nodes
    nn.ReLU(),
    nn.Linear(32, 16),
    nn.ReLU(),
    nn.Linear(16, 1),
    nn.Sigmoid()
).to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
  train_loss = train_loop(train_loader, model, optimizer, criterion, device)
  test_loss = test_loop(test_loader, model, criterion, device)

  print(f"[Epoch {epoch+1}/{num_epochs}] TrainLoss: {train_loss}; TestLoss: {test_loss}")
  wandb.log({
    "train_loss": train_loss,
    "test_loss": test_loss
  })

wandb.finish()

<ipython-input-6-508fe56c84fc>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-6-508fe56c84fc>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

[Epoch 1/10] TrainLoss: 0.6622954627101341; TestLoss: 0.5792353802553102
[Epoch 2/10] TrainLoss: 0.6088828290446421; TestLoss: 0.5528384490386068
[Epoch 3/10] TrainLoss: 0.5903971067975077; TestLoss: 0.5310227717767214
[Epoch 4/10] TrainLoss: 0.5809864361634415; TestLoss: 0.5159992725489526
[Epoch 5/10] TrainLoss: 0.5643915905711356; TestLoss: 0.49290778000927504
[Epoch 6/10] TrainLoss: 0.5251861556192462; TestLoss: 0.4763176189787561
[Epoch 7/10] TrainLoss: 0.49219536781311035; TestLoss: 0.575100604715294
[Epoch 8/10] TrainLoss: 0.5168125689699409; TestLoss: 0.4816162268209724
[Epoch 9/10] TrainLoss: 0.46607794219188475; TestLoss: 0.4658387656318409
[Epoch 10/10] TrainLoss: 0.47976387283775246; TestLoss: 0.4577245062955931


test_loss,█▆▅▄▃▂█▂▁▁
train_loss,█▆▅▅▅▃▂▃▁▁
test_loss,0.45772
train_loss,0.47976


# Artifacts (Data Registry + Model Registry)
First Model Artifacts - Raw Titanic Data

In [4]:
run = wandb.init(project="Artifacts Registry", job_type="data-loading")

# Save data locally
raw_titanic_df = load_titanic(cleaned=False, Xy=False)
raw_titanic_df.to_csv('raw_titanic.csv', index=False)  # Save as CSV

# Create Artifact object
raw_dataset_artifact = wandb.Artifact(name='raw_titanic', type='dataset')

# Add files to the artifact (multiple)
raw_dataset_artifact.add_file('raw_titanic.csv')

# Log the artifact
wandb.log_artifact(raw_dataset_artifact, aliases=["raw"])

wandb.finish()

## Add Cleaned Dataset Artifact (version 0)

In [5]:
run = wandb.init(project="Artifacts Registry", job_type="data-loading")

# Get clean data
clean_titanic_df_dropped = clean_titanic(raw_titanic_df, fill_missing=False)
clean_titanic_df_dropped.to_csv('clean_titanic.csv', index=False)

# Create and Log New Artifact
run.log_artifact(
    artifact_or_path="clean_titanic.csv",
    name="clean_titanic",
    type="dataset",
    aliases=["dropped"]
)

wandb.finish()

### Create New Cleaned Dataset (version 1)

In [14]:
run = wandb.init(project="Artifacts Registry", job_type="data-loading")

# Create new artifact version
artifact_name = "clean_titanic"
artifact_old = run.use_artifact(
    f'ayenyeinsan2904-chiang-mai-university/Artifacts Registry/{artifact_name}:latest',
    type='dataset'
)
artifact_next_version = wandb.Artifact(artifact_name, type='dataset')

# Update the data or get new data for the next version
## artifact_old_path = artifact_old.download()
## clean_titanic_df_old = pd.read_csv(artifact_old_path)
clean_titanic_df_filled = clean_titanic(raw_titanic_df, fill_missing=True)
clean_titanic_df_filled.to_csv('clean_titanic.csv', index=False)

# Log the new artifact
artifact_next_version.add_file('clean_titanic.csv')
wandb.log_artifact(artifact_next_version, aliases =["filled"])

wandb.finish()

<ipython-input-3-508fe56c84fc>:23: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Age'].fillna(df['Age'].median(), inplace=True)
<ipython-input-3-508fe56c84fc>:24: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try us

## Create Model Artifact

In [16]:
run = wandb.init(project="Artifacts Registry", job_type="training")

# Load artifact dataset
artifact = run.use_artifact(artifact_or_name=
                  f"ayenyeinsan2904-chiang-mai-university/Artifacts Registry/clean_titanic:latest")
path = artifact.download()
df = pd.read_csv(f"{path}/clean_titanic.csv")

# Get feature and labels
X, y = df.drop('Survived', axis=1), df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

# Create a model and save it
clf = RandomForestClassifier(n_estimators=1, max_depth=5)
clf.fit(X_train, y_train)
wandb.log({"accuracy": clf.score(X_test, y_test)})

# Create Artifact object
joblib.dump(clf, 'random_forest_model.pkl')
model_artifact = wandb.Artifact(name='random_forest_model', type='model')
model_artifact.add_file('random_forest_model.pkl')
wandb.log_artifact(model_artifact)

wandb.finish()

wandb:   1 of 1 files downloaded.  


accuracy,▁
accuracy,0.77095


## Create a table

In [19]:
run = wandb.init(project='Table')

# Load artifact dataset
artifact = run.use_artifact(
    artifact_or_name=f"ayenyeinsan2904-chiang-mai-university/Artifacts Registry/clean_titanic:latest"
)
path = artifact.download()
df = pd.read_csv(f"{path}/clean_titanic.csv")

# Get feature and labels
X, y = df.drop('Survived', axis=1), df['Survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
                                                    random_state=42)

# Create a model and save it
model_artifact = run.use_artifact(
    'ayenyeinsan2904-chiang-mai-university/Artifacts Registry/random_forest_model:v0',
    type='model'
)
model_artifact_dir = model_artifact.download()
clf = joblib.load(f"{model_artifact_dir}/random_forest_model.pkl")
wandb.log({"accuracy": clf.score(X_test, y_test)})

# Create Table
table = wandb.Table(columns=['y_pred', 'y_true', 'prob_survive',
                             'Pclass', 'Sex', 'Age',
                             'SibSp', 'Parch', 'Fare',
                             'Embarked'])

# Add prediction data to the table
for i in range(len(X_test)):
  # Get Values
  row = X_test.iloc[[i]]
  y_pred = clf.predict(row)[0]
  y_prob_1 = clf.predict_proba(row)[0][1]
  y_true = y_test.iloc[i]

  # Add values to table
  table.add_data(
      y_pred, y_true, y_prob_1,
      row['Pclass'].iloc[0], row['Sex'].iloc[0], row['Age'].iloc[0],
      row['SibSp'].iloc[0], row['Parch'].iloc[0], row['Fare'].iloc[0],
      row['Embarked'].iloc[0])

run.log({"test_predictions": table})

wandb.finish()

wandb:   1 of 1 files downloaded.  
wandb:   1 of 1 files downloaded.  


accuracy,▁
accuracy,0.77095


# Sweep

In [20]:
# def main():
#   run = wandb.init()
#   config = wandb.config

#   # Get Train and Test Data
#   artifact = run.use_artifact(
#       artifact_or_name=f"ayenyeinsan2904-chiang-mai-university/Artifacts Registry/clean_titanic:latest"
#   )
#   path = artifact.download()
#   df = pd.read_csv(f"{path}/clean_titanic.csv")
#   X, y = df.drop('Survived', axis=1), df['Survived']
#   X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,
#                                                       random_state=42)

#   # Get PyTorch Data Loaders w/ batch_size
#   train_loader, test_loader = get_data_loaders(X_train, X_test, y_train, y_test,
#                                                batch_size=config.batch_size)

#   # Define Simple PyTorch Model
#   model = nn.Sequential(
#       nn.Linear(7, 32), # 7 input features -> 32 hidden nodes
#       nn.ReLU(),
#       nn.Linear(32, 16),
#       nn.ReLU(),
#       nn.Linear(16, 1),
#       nn.Sigmoid()
#   ).to(device)

#   # Loss Funciton
#   criterion = nn.BCELoss()

#   # Optimizer
#   if config.optimizer == 'adam':
#     optimizer = optim.Adam(model.parameters(), lr=config.learning_rate)
#   else:
#     optimizer = optim.SGD(model.parameters(), lr=config.learning_rate)

#   for epoch in range(config.num_epochs):
#     train_loss = train_loop(train_loader, model, optimizer, criterion, device)
#     test_loss = train_loop(test_loader, model, optimizer, criterion, device)

#     wandb.log({
#       "train_loss": train_loss,
#       "test_loss": test_loss
#     })

#   wandb.finish()

# # Initialize the sweep
# sweep_config = {
#     "method": "random",
#     "metric": {"goal": "minimize", "name": "test_loss"},
#     "parameters": {
#         "learning_rate": {"min": 0.001, "max": 0.1},
#         "batch_size": {"values": [32, 64, 128]},
#         "optimizer": {"values": ["adam", "sgd"]},
#         "num_epochs": {"values": [5, 10, 15]}
#     },
# }
# sweep_id = wandb.sweep(sweep_config, project='Sweep')

# # Run the sweep
# wandb.agent(sweep_id, function=main)

Create sweep with ID: nmaceo43
Sweep URL: https://wandb.ai/ayenyeinsan2904-chiang-mai-university/Sweep/sweeps/nmaceo43


wandb: Agent Starting Run: 8avtp6z8 with config:
wandb: 	batch_size: 128
wandb: 	learning_rate: 0.06957487949218227
wandb: 	num_epochs: 5
wandb: 	optimizer: sgd


wandb:   1 of 1 files downloaded.  


test_loss,█▂▃▁▃
train_loss,█▁▁▂▁
test_loss,0.57386
train_loss,0.61797


wandb: Agent Starting Run: n4hqtqd5 with config:
wandb: 	batch_size: 64
wandb: 	learning_rate: 0.08587487420462472
wandb: 	num_epochs: 15
wandb: 	optimizer: adam


...

test_loss,█▇▆▄▄▁▂▁▁▂
train_loss,█▂▂▂▁▁▁▁▁▁
test_loss,0.49048
train_loss,0.49478


wandb: Agent Starting Run: 9ww9lfd6 with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.02428270229801251
wandb: 	num_epochs: 10
wandb: 	optimizer: adam


wandb:   1 of 1 files downloaded.  


test_loss,█▅▄▂▄▃▁▁▂▁
train_loss,█▄▃▂▂▃▁▂▁▁
test_loss,0.45442
train_loss,0.48375


wandb: Agent Starting Run: l4jptkzp with config:
wandb: 	batch_size: 32
wandb: 	learning_rate: 0.06489965458573803
wandb: 	num_epochs: 15
wandb: 	optimizer: sgd
wandb: Ctrl + C detected. Stopping sweep.
